In [1]:
from pandas import read_csv
import numpy as np
import pandas as pd
import datetime

input_window_length = 168
output_window_length = 24
window_length = input_window_length + output_window_length
n_i = 370
stride_size = 24

df = pd.read_csv('LD2011_2014.txt',sep=';',parse_dates=[0],decimal=',')

df.shape

df.rename(columns={"Unnamed: 0":"date"},inplace=True)
df['day'] = df.date.apply(lambda x: datetime.datetime.strftime(x, format="%Y-%m-%d"))
df['hour'] = df.date.dt.hour
del df['date']

df.info()

df_hourly = df.groupby(['day','hour']).sum().reset_index()
df_hourly.day = pd.to_datetime(df_hourly.day)

df_hourly.head()

dataset = df_hourly.copy()
#dataset = df.groupby(['day','hour']).sum().reset_index()
#df1.day = pd.to_datetime(df1.day)

del dataset['day']
del dataset['hour']

dataset.isnull().sum()

dataset.shape

from newer_preprocess import reframe

values = dataset.values

values.shape

reframed = reframe(values, input_window_length, output_window_length, stride_size, n_i).astype('float32')

reframed.shape

reframed

v_i_g = np.asarray([ [np.mean(reframed[i,:,0]) + 1] for i in range(reframed.shape[0])])

v_i_g.shape

reframed[:,:,0] = (reframed[:,:,0] / v_i_g)
reframed[:,:,5] = (reframed[:,:,5] / v_i_g)


assert reframed.shape[0] == v_i_g.shape[0]


order = np.arange(0,reframed.shape[0],1)

order

np.random.shuffle(order)

reframed = reframed[order,:,:]
v_i_g = v_i_g[order,:]

np.save('vi-g-all.npy', v_i_g)

np.save('reframed-data-all.npy', reframed)